# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Салынский Александр Викторович`

Группа: `DS-12`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [287]:
from collections import OrderedDict

from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.data import Data, ALL
from shapely.geometry import Point
import osmapi as osm
from random import uniform
from geopy.distance import distance, lonlat
import re
import geopandas as gpd
import folium
import pandas as pd

api = osm.OsmApi(username='geo-course', 
                 password='5b3ce3597851110001cf6248053f97de5fd74d829d736d9dfa6d8fe4')

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
polygon_afg = world.loc[world.iso_a3 == 'AFG'].geometry.values[0]

In [87]:
def find_min_dist(poly, point) -> float:
    min_dist = 45_000
    for coord in poly.exterior.coords:
        min_dist = min(min_dist, distance(coord, (point.x, point.y)))
    return min_dist

used_points = []

points = [{}, {}] 
points[0]['point'] = Point(-90, -180)
points[1]['point'] = Point(90, 180)

found = False
if polygon_afg.contains(points[0]['point']):
    found = True
    point = points[0]['point']
if polygon_afg.contains(points[1]['point']):
    found = True
    point = points[1]['point']

points[0]['dist'] = find_min_dist(polygon_afg, points[0]['point'])
points[1]['dist'] = find_min_dist(polygon_afg, points[1]['point'])

used_points.append(points[0]['point'])
used_points.append(points[1]['point'])

while not found:
    p_upd = 0
    if points[0]['dist'] < points[1]['dist']:
        p_upd = 1
        
    point_lat = Point((points[0]['point'].x + points[1]['point'].x) / 2, points[p_upd]['point'].y)
    point_lon = Point(points[p_upd]['point'].x, (points[0]['point'].y + points[1]['point'].y) / 2)
    dist_lon = find_min_dist(polygon_afg, point_lon)
    dist_lat = find_min_dist(polygon_afg, point_lat)
    
    points[p_upd]['point'] = point_lon
    points[p_upd]['dist'] = dist_lon
    if dist_lon > dist_lat:
        points[p_upd]['point'] = point_lat
        points[p_upd]['dist'] = dist_lat
        
    if polygon_afg.contains(points[p_upd]['point']):
        found = True
        point = points[p_upd]['point']
        
    used_points.append(points[p_upd]['point'])

In [88]:
nominatim = Nominatim()
area_id_af = nominatim.query('Afganistan').areaId()

overpass = Overpass()
query = overpassQueryBuilder(area=area_id_af, elementType='node', selector='"place"="country"', out='body')
center_node = overpass.query(query).nodes()[0]

print(f'Central coordinates for AFG: {center_node.lon(), center_node.lat()}')

Central coordinates for AFG: (66.2385139, 33.7680065)


In [93]:
m = folium.Map((center_node.lat(), center_node.lon()), 
               zoom_start=1, 
               tiles='Stamen Toner')

tooltip = 'Check step'

for i, point in enumerate(used_points):
    folium.Marker([point.y, point.x], 
                  popup=f'<i>Point received on step: {i}</i>', 
                  tooltip=tooltip,
                  icon=folium.Icon(color='red', icon='info-sign')).add_to(m)
m

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [117]:
result = overpass.query('relation["admin_level"="2"][boundary=administrative]; out;')

pattern = re.compile(r' -|— ')
countries = []

for rel in result.relations():
    if not pattern.findall(rel.tag('name:en')) and not rel.id() % 10:
        countries.append(rel)

In [219]:
def count_residental_buildings(country) -> int:
    query = overpassQueryBuilder(area=country.id() + 3_600_000_000,
                                 elementType=['relation', 'way'],
                                 selector=[f'"building"="residential"',
                                           f'"building"="apartments"',
                                           f'"building"="house"'],
                                 out='count')
    return overpass.query(query, timeout=60).countElements()

dimensions = OrderedDict({
    'country': OrderedDict({
        country.tag('name:en'): country
        for country in countries
    })
})

data = Data(count_residental_buildings, dimensions)

country_1 = data.getDataFrame().sort_values('value', ascending=False).iloc[0].name
country_1

In [273]:
# Here I just failed - I have tried to use banks, ATM's and so on with different areas, but no luck.

country_1_id = nominatim.query(country_1).areaId()
query = overpassQueryBuilder(area=country_1_id + 3_600_000_000,
                             elementType=['node', 'way'],
                             selector=[f'"amenity"="atm"'],
                             out='body')
res = overpass.query(query, timeout=60)

[nominatim] downloading data: search
[overpass] downloading data: [timeout:60][out:json];area(7200065606)->.searchArea;(node["amenity"="atm"](area.searchArea);way["amenity"="atm"](area.searchArea);); out body;


In [292]:
nominatim = Nominatim()

def get_capital_id(country) -> int:
    area_id = nominatim.query(country.tag('name:en')).areaId()
    relation = api.RelationGet(f'{area_id - 3_600_000_000}')
    
    # Taiwan is out of admin_center via API, however, via OSM site they have it :thinking:
    node_id = {
        'Taiwan': 1147314253,
        'Singapore': area_id - 3_600_000_000
    }.get(country.tag('name:en'), -1)
    if node_id == -1:
        node_id = list(filter(lambda member: member['role'] == 'admin_centre', relation['member']))[0]['ref']
    return node_id

pharmacy_names = pd.read_csv('../data/pharmacy.csv').Translation
def count_pharmacy(capital_id) -> int:
    query = overpassQueryBuilder(area=capital_id + 3_600_000_000,
                                 elementType=['relation', 'way'],
                                 selector=[f'"amenity"="{name}"' for name in pharmacy_names],
                                 out='count')
    return overpass.query(query, timeout=60).countElements()

dimensions_pharmacy = OrderedDict({
    'country': OrderedDict({
        country.tag('name:en'): get_capital_id(country)
        for country in countries
    })
})

data_pharmacy = Data(count_pharmacy, dimensions_pharmacy)

[overpass] downloading data: [timeout:60][out:json];area(3634567423)->.searchArea;(relation["amenity"="صيدلية"]["amenity"="farmacia"]["amenity"="farmàcia"]["amenity"="lékárna"]["amenity"="apotek"]["amenity"="apotheek"]["amenity"="pharmacy"]["amenity"="apteekki"]["amenity"="pharmacie"]["amenity"="Apotheke"]["amenity"="gyógyszertár"]["amenity"="farmacia"]["amenity"="薬局"]["amenity"="apteka"]["amenity"="farmácia"]["amenity"="аптека"]["amenity"="lekáreň"]["amenity"="farmacia"]["amenity"="apotek"]["amenity"="eczane"]["amenity"="аптека"]["amenity"="tiệm"](area.searchArea);way["amenity"="صيدلية"]["amenity"="farmacia"]["amenity"="farmàcia"]["amenity"="lékárna"]["amenity"="apotek"]["amenity"="apotheek"]["amenity"="pharmacy"]["amenity"="apteekki"]["amenity"="pharmacie"]["amenity"="Apotheke"]["amenity"="gyógyszertár"]["amenity"="farmacia"]["amenity"="薬局"]["amenity"="apteka"]["amenity"="farmácia"]["amenity"="аптека"]["amenity"="lekáreň"]["amenity"="farmacia"]["amenity"="apotek"]["amenity"="eczane"]

[overpass] downloading data: [timeout:60][out:json];area(3627565076)->.searchArea;(relation["amenity"="صيدلية"]["amenity"="farmacia"]["amenity"="farmàcia"]["amenity"="lékárna"]["amenity"="apotek"]["amenity"="apotheek"]["amenity"="pharmacy"]["amenity"="apteekki"]["amenity"="pharmacie"]["amenity"="Apotheke"]["amenity"="gyógyszertár"]["amenity"="farmacia"]["amenity"="薬局"]["amenity"="apteka"]["amenity"="farmácia"]["amenity"="аптека"]["amenity"="lekáreň"]["amenity"="farmacia"]["amenity"="apotek"]["amenity"="eczane"]["amenity"="аптека"]["amenity"="tiệm"](area.searchArea);way["amenity"="صيدلية"]["amenity"="farmacia"]["amenity"="farmàcia"]["amenity"="lékárna"]["amenity"="apotek"]["amenity"="apotheek"]["amenity"="pharmacy"]["amenity"="apteekki"]["amenity"="pharmacie"]["amenity"="Apotheke"]["amenity"="gyógyszertár"]["amenity"="farmacia"]["amenity"="薬局"]["amenity"="apteka"]["amenity"="farmácia"]["amenity"="аптека"]["amenity"="lekáreň"]["amenity"="farmacia"]["amenity"="apotek"]["amenity"="eczane"]

[overpass] downloading data: [timeout:60][out:json];area(4747314253)->.searchArea;(relation["amenity"="صيدلية"]["amenity"="farmacia"]["amenity"="farmàcia"]["amenity"="lékárna"]["amenity"="apotek"]["amenity"="apotheek"]["amenity"="pharmacy"]["amenity"="apteekki"]["amenity"="pharmacie"]["amenity"="Apotheke"]["amenity"="gyógyszertár"]["amenity"="farmacia"]["amenity"="薬局"]["amenity"="apteka"]["amenity"="farmácia"]["amenity"="аптека"]["amenity"="lekáreň"]["amenity"="farmacia"]["amenity"="apotek"]["amenity"="eczane"]["amenity"="аптека"]["amenity"="tiệm"](area.searchArea);way["amenity"="صيدلية"]["amenity"="farmacia"]["amenity"="farmàcia"]["amenity"="lékárna"]["amenity"="apotek"]["amenity"="apotheek"]["amenity"="pharmacy"]["amenity"="apteekki"]["amenity"="pharmacie"]["amenity"="Apotheke"]["amenity"="gyógyszertár"]["amenity"="farmacia"]["amenity"="薬局"]["amenity"="apteka"]["amenity"="farmácia"]["amenity"="аптека"]["amenity"="lekáreň"]["amenity"="farmacia"]["amenity"="apotek"]["amenity"="eczane"]

[overpass] downloading data: [timeout:60][out:json];area(3620981158)->.searchArea;(relation["amenity"="صيدلية"]["amenity"="farmacia"]["amenity"="farmàcia"]["amenity"="lékárna"]["amenity"="apotek"]["amenity"="apotheek"]["amenity"="pharmacy"]["amenity"="apteekki"]["amenity"="pharmacie"]["amenity"="Apotheke"]["amenity"="gyógyszertár"]["amenity"="farmacia"]["amenity"="薬局"]["amenity"="apteka"]["amenity"="farmácia"]["amenity"="аптека"]["amenity"="lekáreň"]["amenity"="farmacia"]["amenity"="apotek"]["amenity"="eczane"]["amenity"="аптека"]["amenity"="tiệm"](area.searchArea);way["amenity"="صيدلية"]["amenity"="farmacia"]["amenity"="farmàcia"]["amenity"="lékárna"]["amenity"="apotek"]["amenity"="apotheek"]["amenity"="pharmacy"]["amenity"="apteekki"]["amenity"="pharmacie"]["amenity"="Apotheke"]["amenity"="gyógyszertár"]["amenity"="farmacia"]["amenity"="薬局"]["amenity"="apteka"]["amenity"="farmácia"]["amenity"="аптека"]["amenity"="lekáreň"]["amenity"="farmacia"]["amenity"="apotek"]["amenity"="eczane"]

In [293]:
data_pharmacy


                          value
country                        
Brazil                        0
Estonia                       0
Australia                     0
Eswatini                      0
Bangladesh                    0
Syria                         0
Liberia                       0
Central African Republic      0
Ethiopia                      0
Cameroon                      0
Tanzania                      0
Malawi                        0
Uzbekistan                    0
Honduras                      0
Portugal                      0
Azerbaijan                    0
Taiwan                        0
Comoros                       0
São Tomé and Príncipe         0
Singapore                     0
Antigua and Barbuda           0
Kosovo                        0
Brunei                        0
Tokelau                       0
Norway                        0

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [ ]:
# put your code here